# Visual proprioception flow

Create the full flow for training models for visual proprioception. This notebook programmatically generates a set of exp/runs that cover all the necessary components for a visual proprioception system (sensor processing,  visual proprioception regressor and verification notebooks).

Then, it writes the exp/runs into an external directory full separated from the github source, and creates an automation script that runs them. A separate directory for the results is also created. 

Finally, it runs the necessary notebooks to execute the whole flow using papermill.

The results directory contain the output of this flow, both in terms of trained models, as well as results (in the verification exp/run).

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import yaml
import tqdm
import papermill
# from automate import automate_exprun
import visproprio_helper 
from demonstration.demonstration import list_demos

In [2]:
# Setting up a separate directory for generated exp/run config files and the results
hostname = socket.gethostname()
print(f"***Hostname is {hostname}")
if hostname == "raven":
    rootdir = pathlib.Path(f"~/WORK/_DataExternal/").expanduser()
elif hostname == "szenes.local":
    rootdir = pathlib.Path(f"~/Documents/Develop/Data/").expanduser()
elif hostname == "glassy":
    rootdir = pathlib.Path(f"~/Work/_DataExternal/").expanduser()
else:
    rootdir = pathlib.Path(Config()["experiment_external"])
visproprio_helper.external_setup("VisualProprioception_flow_00", rootdir)


***Hostname is szenes.local
***Path for external experiments:
/Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/exprun
***Path for external data:
/Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/result
***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
***ExpRun**: Experiment config path changed to /Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/exprun
***ExpRun**: Experiment data path changed to /Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/result
***ExpRun**: Experiment demonstration copied to /Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/exprun/demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to /Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/e

(PosixPath('/Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/exprun'),
 PosixPath('/Users/lboloni/Documents/Develop/Data/VisualProprioception_flow_00/result'))

In [ ]:
def generate_sensorprocessing_conv_vae(exprun_path, result_path, params):
    """Generate the experiment for the training of the conv-vae sensorprocessing with the right training data and parameters. Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. 
    NOTE: a similar function is in Flow_BehaviorCloning.
    """
    val = {}
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_sp"], params["run_sp"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "Train_SP_Conv-VAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    vparams = {}
    vparams["experiment"] = params["exp_sp"]
    vparams["run"] = params["run_sp"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_sp"], "runname": params["run_sp"], "automation_entry": v}

In [ ]:
def generate_sensorprocessing_propriotuned_cnn(exprun_path, result_path, params):
    """Generate the experiment for the training of the propriotuned CNN with the right training data and parameters. 
    Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. 
    """
    val = {}
    val["name"] = params["name"] # ex. resnet-50-128
    val["image_size"] = params["image_size"] # ex [256, 256]    
    val["latent_size"] = params["latent_size"]
    # model architecture
    val["model"] = params["model"] # eg. 


    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_sp"], params["run_sp"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "Train_SP_Conv-VAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    vparams = {}
    vparams["experiment"] = params["exp_sp"]
    vparams["run"] = params["run_sp"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_sp"], "runname": params["run_sp"], "automation_entry": v}    